In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn.functional as F
import os 
import torch
import torch.nn as nn
import time
import pandas as pd
from sklearn import manifold, datasets
from sklearn.metrics import precision_recall_fscore_support, roc_curve, confusion_matrix
import glob
import itertools

## Load Data

In [ ]:
y_pred = np.load("y_pred_val_1440chunks_21_everything_9.npy")
y_true = np.load("y_true_val_1440chunks_21_everything_9.npy")
tics = np.load("y_tic_val_1440chunks_21_everything_9.npy")
secs = np.load("y_sec_val_1440chunks_21_everything_9.npy")

mask = ~np.isnan(y_pred).any(axis=1)
y_pred = y_pred[mask]
y_true = y_true[mask]
tics = tics[mask]
secs = secs[mask]


files = glob.glob("top_1000_tic_ids/*")

# y_pred2 = np.load("y_pred_val_1440chunks_21_everything_10.npy")
# y_true2 = np.load("y_true_val_1440chunks_21_everything_10.npy")
# tics2 = np.load("y_tic_val_1440chunks_21_everything_10.npy")


In [ ]:
np.shape(y_true)

## ROC + stats

In [ ]:
labels_new_val = y_true[:,:-1].copy()
labels_new_val[:,-1] += y_true[:,-1].copy()

# labels_new_val2 = y_true2[:,:-1].copy()
# labels_new_val2[:,-1] += y_true2[:,-1].copy()

print(confusion_matrix(np.argmax(labels_new_val,axis=1), np.argmax(y_pred,axis=1)))

print(precision_recall_fscore_support(np.argmax(labels_new_val,axis=1), np.argmax(y_pred,axis=1), average=None))

a, b, c = roc_curve(labels_new_val[:,1], y_pred[:,1])
#a2, b2, c2 = roc_curve(labels_new_val2[:,2], y_pred2[:,2])
#a2, b2, c2 = roc_curve(y_true[:,0], y_pred2[:,0])

plt.figure()
plt.plot(a,b,label="9")
#plt.plot(a2,b2,label="10")
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nEB vs the rest")
plt.legend()
#plt.savefig("ROC_fluxbkg_vs_fluxonly.png")


thr = 400

other = y_pred[y_pred[:,1]<c[thr]]
eb = y_pred[y_pred[:,1]>=c[thr]]


correct = 0
cont = 0
ebs = 0
missed = 0 
others = 0
pcs_in_eb = 0

for i in range(len(y_pred)):
  if y_pred[i,1] < c[thr] and y_true[i,1] == 0:
    correct = correct + 1
    others = others + 1
  elif y_pred[i,1] >= c[thr] and y_true[i,1] == 1:
    correct = correct + 1
    ebs = ebs + 1
  elif y_pred[i,1] >= c[thr] and y_true[i,1] == 0:
    cont = cont + 1
    if y_true[i,0] == 1:
        pcs_in_eb = pcs_in_eb + 1
    
  elif y_pred[i,1] < c[thr] and y_true[i,1] == 1:
    missed = missed + 1


accuracy = ebs/len(y_true[y_true[:,1]==1])
contamination = cont/(len(eb))
print(f"percent of ebs found: {accuracy}, contamination: {contamination}, ebs found: {ebs}, ebs missed: {missed}, false positives: {cont}")

print(f"% of FP and PCs removed: {(others)*100/(len(y_true[y_true[:,0]==0]))}")
print(f"num removed: {others}")
print(f"% planets discarded as ebs: {pcs_in_eb/len(y_true[y_true[:,0]==1])}")
print(f"planets discarded as ebs: {pcs_in_eb}")

## Unique tics EB test

In [2]:
y_pred = np.load("y_pred_val_16.npy")
y_true = np.load("y_true_val_16.npy")
tics = np.load("y_tic_val_16.npy")

y_pred2 = np.load("y_pred_val_17.npy")
y_true2 = np.load("y_true_val_17.npy")
tics2 = np.load("y_tic_val_17.npy")

y_pred3 = np.load("y_pred_val_18.npy")
y_true3 = np.load("y_true_val_18.npy")
tics3 = np.load("y_tic_val_18.npy")


y_pred4 = np.load("y_pred_val_19.npy")
y_true4 = np.load("y_true_val_19.npy")
tics4 = np.load("y_tic_val_19.npy")



In [3]:
y_pred = np.load("y_pred_test_16.npy")
y_true = np.load("y_true_test_16.npy")
tics = np.load("y_tic_test_16.npy")
secs = np.load("y_sec_test_16.npy")

y_pred2 = np.load("y_pred_test_17.npy")
y_true2 = np.load("y_true_test_17.npy")
tics2 = np.load("y_tic_test_17.npy")
secs2 = np.load("y_sec_test_17.npy")

y_pred3 = np.load("y_pred_test_18.npy")
y_true3 = np.load("y_true_test_18.npy")
tics3 = np.load("y_tic_test_18.npy")
secs3 = np.load("y_sec_test_18.npy")

y_pred4_test = np.load("y_pred_test_19.npy")
y_true4_test = np.load("y_true_test_19.npy")
tics4_test = np.load("y_tic_test_19.npy")
secs4_test = np.load("y_sec_test_19.npy")


y_pred5_test = np.load("y_pred_test_s49_19.npy")
y_true5_test = np.load("y_true_test_s49_19.npy")
tics5_test = np.load("y_tic_test_s49_19.npy")
secs5_test = np.load("y_sec_test_s49_19.npy")

In [ ]:
plt.figure()

plt.plot(roc_curve(y_true[:,1], y_pred[:,1])[0],roc_curve(y_true[:,1], y_pred[:,1])[1],label=1)
plt.plot(roc_curve(y_true2[:,1], y_pred2[:,1])[0],roc_curve(y_true2[:,1], y_pred2[:,1])[1],label=2)
plt.plot(roc_curve(y_true3[:,1], y_pred3[:,1])[0],roc_curve(y_true3[:,1], y_pred3[:,1])[1],label=3)
plt.plot(roc_curve(y_true4[:,1], y_pred4[:,1])[0],roc_curve(y_true4[:,1], y_pred4[:,1])[1],label=4)
plt.legend()

In [ ]:
labels_new_val = y_true4[:,:-1].copy()
labels_new_val[:,-1] += y_true4[:,-1].copy()

# labels_new_val = y_true[:,:-2].copy()
# labels_new_val[:,-1] += y_true[:,-1].copy()
# labels_new_val[:,-1] += y_true[:,-2].copy()


# y_pred_new = y_pred[:,:-1].copy()
# y_pred_new[:,-1] += y_pred[:,-1].copy()


a, b, c = roc_curve(labels_new_val[:,1], y_pred4[:,1])

cutoff = 9
top_cut = 500

my_score = []
pcs_rems = []
#pcs_intop = []
max_score = -10000
for thr in c:
    #for thr2 in c2:

    #y_pred4_cut = y_pred4[:top_cut]
    #labels_new_val = labels_new_val[:top_cut]
    #tics_new = tics4[:top_cut]
    
    pc_found_mask = np.where((y_pred4[:,1]<thr)&(labels_new_val[:,0] == 1))
    pc_found_tics = np.unique(tics4[pc_found_mask])
    num_pc_found = len(pc_found_tics)
    
    pc_missed_mask = np.where((y_pred4[:,1]>=thr)&(labels_new_val[:,0] == 1))
    pc_missed_tics = np.unique(tics4[pc_missed_mask])
    pc_missed_tics = pc_missed_tics[~np.isin(pc_missed_tics,pc_found_tics)]
    num_pc_missed = len(pc_missed_tics)
    
    other_found_mask = np.where((labels_new_val[:,0] == 0)&(y_pred4[:,1]>=thr))
    other_found_tics = np.unique(tics4[other_found_mask])
    other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_found_tics)]
    
    #num_pc_missed = num_pc_missed + len(other_found_tics[np.isin(other_found_tics,pc_missed_tics)])
    
    other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_missed_tics)]
    num_other_found = len(other_found_tics)
    
    other_missed_mask = np.where((labels_new_val[:,0] == 0)&(y_pred4[:,1]<thr))
    other_missed_tics = np.unique(tics4[other_missed_mask])
    other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,np.concatenate((pc_found_tics,other_found_tics,pc_missed_tics)))]
    #other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,other_found_tics)]
    #other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,pc_missed_tics)]
    
    num_other_missed = len(other_missed_tics)
    
    
    ##ebs = len(np.where((y_pred4[:,1] >= thr)&(labels_new_val[:,1] == 1))[0])
    #missed = len(np.where((y_pred4[:,1] < thr)&(labels_new_val[:,1] == 1))[0])
    #pcs_in_eb = len(np.where((y_pred4[:,1] >= thr)&(labels_new_val[:,0] == 1))[0])
    
    #pcs_found = len(np.where((y_pred4_cut[:,1]<thr)&(labels_new_val[:,0] == 1))[0])
    #others_found = len(np.where((labels_new_val[:,0] == 0))[0])
    #pcs_missed = len(np.where((labels_new_val[:,0] == 1)&(y_pred4[:,1]>=thr))[0])
    
    
    #score = pcs_found/len(np.where(labels_new_val[:,0] == 1)[0])-others_found/len(np.where(labels_new_val[:,0] == 0)[0])
    #score = ebs/len(np.where(labels_new_val[:,1] == 1)[0])-(pcs_in_eb/len(np.where(labels_new_val[:,0] == 1)[0]))#-(missed/len(np.where(labels_new_val[:,1] == 1)[0]))
    #score = pcs_found
    score = (num_pc_found - 5*num_pc_missed)/len(np.where(labels_new_val[:,0] == 1)[0]) + num_other_found/len(np.where(labels_new_val[:,1] == 1)[0])
    my_score.append(score)
    pcs_rems.append(num_pc_missed)
    #pcs_intop.append(num_pc_found)
    if score > max_score:
        max_score = score
        best_c = thr
        best_pc = num_pc_found
        best_other = num_other_found
        #best_ebs = ebs
        best_missed = num_pc_missed
        #best_pcs_in_eb = pcs_in_eb
        #best_pcs_found = pcs_found
        best_other_missed = num_other_missed


    #plt.plot(thr,ebs-pcs_in_eb-missed,".")

    
plt.figure()
plt.plot(pcs_rems,my_score)
plt.xlabel("PCs removed")
plt.ylabel("Score")
#plt.vlines(0,0,0.4,linestyles="dashed",color = "red")
#plt.vlines(cutoff,0,0.4,linestyles="dashed",color = "red")
#print(f"found ebs: {best_ebs}, missed ebs: {best_missed}, pcs removed: {best_pcs_in_eb},pcs in 500:{best_pcs_found},others in 500: {best_other_found}")    
print(f"pcs found:{best_pc},others found: {best_other}, pcs missed: {best_missed}, others missed: {best_other_missed}")  
print(best_pc+best_other+best_missed+best_other_missed)


labels_new_val_test = y_true4_test[:,:-1].copy()
labels_new_val_test[:,-1] += y_true4_test[:,-1].copy()

pc_found_mask = np.where((y_pred4_test[:,1]<best_c)&(labels_new_val_test[:,0] == 1))
pc_found_tics = np.unique(tics4_test[pc_found_mask])
num_pc_found = len(pc_found_tics)

pc_missed_mask = np.where((y_pred4_test[:,1]>=best_c)&(labels_new_val_test[:,0] == 1))
pc_missed_tics = np.unique(tics4_test[pc_missed_mask])
pc_missed_tics = pc_missed_tics[~np.isin(pc_missed_tics,pc_found_tics)]
num_pc_missed = len(pc_missed_tics)

other_found_mask = np.where((labels_new_val_test[:,0] == 0)&(y_pred4_test[:,1]>=best_c))
other_found_tics = np.unique(tics4_test[other_found_mask])
other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_found_tics)]

#num_pc_missed = num_pc_missed + len(other_found_tics[np.isin(other_found_tics,pc_missed_tics)])

other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_missed_tics)]
num_other_found = len(other_found_tics)

other_missed_mask = np.where((labels_new_val_test[:,0] == 0)&(y_pred4_test[:,1]<best_c))
other_missed_tics = np.unique(tics4_test[other_missed_mask])
other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,np.concatenate((pc_found_tics,other_found_tics,pc_missed_tics)))]
#other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,other_found_tics)]
#other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,pc_missed_tics)]

num_other_missed = len(other_missed_tics)

print(f"pcs found:{num_pc_found},others found: {num_other_found}, pcs missed: {num_pc_missed}, others missed: {num_other_missed}")  
print(num_pc_found+num_other_found+num_pc_missed+num_other_missed)


In [ ]:
len(np.unique(tics4))

In [4]:
def find_cutoff(ypred,ytrue,tics,pc_score = 5, text = "yes"):
    
    labels_new_val = ytrue[:,:-1].copy()
    labels_new_val[:,-1] += ytrue[:,-1].copy()

    a, b, c = roc_curve(labels_new_val[:,1], ypred[:,1])

    my_score = []
    pcs_rems = []
    max_score = -10000
    
    for thr in c:
        
        pc_found_mask = np.where((ypred[:,1]<thr)&(labels_new_val[:,0] == 1))
        pc_found_tics = np.unique(tics[pc_found_mask])
        num_pc_found = len(pc_found_tics)

        pc_missed_mask = np.where((ypred[:,1]>=thr)&(labels_new_val[:,0] == 1))
        pc_missed_tics = np.unique(tics[pc_missed_mask])
        pc_missed_tics = pc_missed_tics[~np.isin(pc_missed_tics,pc_found_tics)]
        num_pc_missed = len(pc_missed_tics)

        other_found_mask = np.where((labels_new_val[:,0] == 0)&(ypred[:,1]>=thr))
        other_found_tics = np.unique(tics[other_found_mask])
        other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_found_tics)]
        other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_missed_tics)]
        num_other_found = len(other_found_tics)

        other_missed_mask = np.where((labels_new_val[:,0] == 0)&(ypred[:,1]<thr))
        other_missed_tics = np.unique(tics[other_missed_mask])
        other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,np.concatenate((pc_found_tics,other_found_tics,pc_missed_tics)))]
        num_other_missed = len(other_missed_tics)
        
        score = (num_pc_found - pc_score*num_pc_missed)/len(np.where(labels_new_val[:,0] == 1)[0]) + num_other_found/len(np.where(labels_new_val[:,1] == 1)[0])
        my_score.append(score)
        pcs_rems.append(num_pc_missed)
        if score > max_score:
            max_score = score
            best_c = thr
            best_pc = num_pc_found
            best_other = num_other_found
            best_missed = num_pc_missed
            best_other_missed = num_other_missed
            best_pc_tics = pc_missed_tics
        
        
        
    if text == "yes":
        print(f"pcs found:{best_pc}, others found: {best_other}, pcs missed: {best_missed}, others missed: {best_other_missed}")
        print(f"pc missed tic: {best_pc_tics}")
        
    return best_c 

def apply_cutoff(ypred,ytrue,tics,cutoff, text = "yes"):
    
    labels_new_val_test = ytrue[:,:-1].copy()
    labels_new_val_test[:,-1] += ytrue[:,-1].copy()

    pc_found_mask = np.where((ypred[:,1]<cutoff)&(labels_new_val_test[:,0] == 1))
    pc_found_tics = np.unique(tics[pc_found_mask])
    num_pc_found = len(pc_found_tics)

    pc_missed_mask = np.where((ypred[:,1]>=cutoff)&(labels_new_val_test[:,0] == 1))
    pc_missed_tics = np.unique(tics[pc_missed_mask])
    pc_missed_tics = pc_missed_tics[~np.isin(pc_missed_tics,pc_found_tics)]
    num_pc_missed = len(pc_missed_tics)

    other_found_mask = np.where((labels_new_val_test[:,0] == 0)&(ypred[:,1]>=cutoff))
    other_found_tics = np.unique(tics[other_found_mask])
    other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_found_tics)]
    other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_missed_tics)]
    num_other_found = len(other_found_tics)

    other_missed_mask = np.where((labels_new_val_test[:,0] == 0)&(ypred[:,1]<cutoff))
    other_missed_tics = np.unique(tics[other_missed_mask])
    other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,np.concatenate((pc_found_tics,other_found_tics,pc_missed_tics)))]
    num_other_missed = len(other_missed_tics)
    
    eb_found_mask = np.where((labels_new_val_test[:,1] == 1)&(ypred[:,1]>=cutoff))
    eb_found_tics = np.unique(tics[eb_found_mask])
    eb_found_tics = eb_found_tics[~np.isin(eb_found_tics,pc_found_tics)]
    eb_found_tics = eb_found_tics[~np.isin(eb_found_tics,pc_missed_tics)]
    num_eb_found = len(eb_found_tics)
    
    if text == "yes":
        print(f"pcs found:{num_pc_found},others found: {num_other_found}, pcs missed: {num_pc_missed}, others missed: {num_other_missed}")  
        print(f"pc missed tic: {pc_missed_tics}")
        print(f"ebs found: {num_eb_found}")
        

In [5]:
c = find_cutoff(y_pred4,y_true4,tics4)

pcs found:221, others found: 762, pcs missed: 1, others missed: 1495
pc missed tic: [422460502]


In [6]:
apply_cutoff(y_pred5_test,y_true5_test,tics5_test,c)

pcs found:228,others found: 222, pcs missed: 1, others missed: 662
pc missed tic: [233182695]
ebs found: 201


In [18]:
def find_cutoff(ypred,ytrue,tics,pc_score = 5, text = "yes"):
    
    labels_new_val = ytrue[:,:-1].copy()
    labels_new_val[:,-1] += ytrue[:,-1].copy()

    a, b, c = roc_curve(labels_new_val[:,0], ypred[:,0])

    my_score = []
    pcs_rems = []
    max_score = -10000
    best_other = 0
    
    for thr in c:
        
        pc_found_mask = np.where((ypred[:,0]>=thr)&(labels_new_val[:,0] == 1))
        pc_found_tics = np.unique(tics[pc_found_mask])
        num_pc_found = len(pc_found_tics)

        pc_missed_mask = np.where((ypred[:,0]<thr)&(labels_new_val[:,0] == 1))
        pc_missed_tics = np.unique(tics[pc_missed_mask])
        pc_missed_tics = pc_missed_tics[~np.isin(pc_missed_tics,pc_found_tics)]
        num_pc_missed = len(pc_missed_tics)

        other_found_mask = np.where((labels_new_val[:,0] == 0)&(ypred[:,0]<thr))
        other_found_tics = np.unique(tics[other_found_mask])
        other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_found_tics)]
        other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_missed_tics)]
        num_other_found = len(other_found_tics)

        other_missed_mask = np.where((labels_new_val[:,0] == 0)&(ypred[:,0]>=thr))
        other_missed_tics = np.unique(tics[other_missed_mask])
        other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,np.concatenate((pc_found_tics,other_found_tics,pc_missed_tics)))]
        num_other_missed = len(other_missed_tics)
        
        #score = (num_pc_found - pc_score*num_pc_missed)/len(np.where(labels_new_val[:,0] == 1)[0]) + num_other_found/len(np.where(labels_new_val[:,1] == 1)[0])
        #my_score.append(score)
        #pcs_rems.append(num_pc_missed)
        #if score > max_score:
        if num_other_found > best_other & num_pc_missed < 2 :
            print("a")
            #max_score = score
            best_c = thr
            best_pc = num_pc_found
            best_other = num_other_found
            best_missed = num_pc_missed
            best_other_missed = num_other_missed
            best_pc_tics = pc_missed_tics
        
        
        
    if text == "yes":
        print(f"pcs found:{best_pc}, others found: {best_other}, pcs missed: {best_missed}, others missed: {best_other_missed}")
        print(f"pc missed tic: {best_pc_tics}")
        
    return best_c 

def apply_cutoff(ypred,ytrue,tics,cutoff, text = "yes"):
    
    labels_new_val_test = ytrue[:,:-1].copy()
    labels_new_val_test[:,-1] += ytrue[:,-1].copy()

    pc_found_mask = np.where((ypred[:,0]>cutoff)&(labels_new_val_test[:,0] == 1))
    pc_found_tics = np.unique(tics[pc_found_mask])
    num_pc_found = len(pc_found_tics)

    pc_missed_mask = np.where((ypred[:,0]<=cutoff)&(labels_new_val_test[:,0] == 1))
    pc_missed_tics = np.unique(tics[pc_missed_mask])
    pc_missed_tics = pc_missed_tics[~np.isin(pc_missed_tics,pc_found_tics)]
    num_pc_missed = len(pc_missed_tics)

    other_found_mask = np.where((labels_new_val_test[:,0] == 0)&(ypred[:,0]<=cutoff))
    other_found_tics = np.unique(tics[other_found_mask])
    other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_found_tics)]
    other_found_tics = other_found_tics[~np.isin(other_found_tics,pc_missed_tics)]
    num_other_found = len(other_found_tics)

    other_missed_mask = np.where((labels_new_val_test[:,0] == 0)&(ypred[:,0]>cutoff))
    other_missed_tics = np.unique(tics[other_missed_mask])
    other_missed_tics = other_missed_tics[~np.isin(other_missed_tics,np.concatenate((pc_found_tics,other_found_tics,pc_missed_tics)))]
    num_other_missed = len(other_missed_tics)
    
    eb_found_mask = np.where((labels_new_val_test[:,1] == 1)&(ypred[:,0]<=cutoff))
    eb_found_tics = np.unique(tics[eb_found_mask])
    eb_found_tics = eb_found_tics[~np.isin(eb_found_tics,pc_found_tics)]
    eb_found_tics = eb_found_tics[~np.isin(eb_found_tics,pc_missed_tics)]
    num_eb_found = len(eb_found_tics)
    
    if text == "yes":
        print(f"pcs found:{num_pc_found},others found: {num_other_found}, pcs missed: {num_pc_missed}, others missed: {num_other_missed}")  
        print(f"pc missed tic: {pc_missed_tics}")
        print(f"ebs found: {num_eb_found}")
        

In [19]:
c = find_cutoff(y_pred4,y_true4,tics4)

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
pcs found:222, others found: 22, pcs missed: 0, others missed: 2235
pc missed tic: []


In [ ]:
apply_cutoff(y_pred5_test,y_true5_test,tics5_test,c)

In [ ]:
labels_new_val = y_true[:,:-1].copy()
labels_new_val[:,-1] += y_true[:,-1].copy()

# labels_new_val = y_true[:,:-2].copy()
# labels_new_val[:,-1] += y_true[:,-1].copy()
# labels_new_val[:,-1] += y_true[:,-2].copy()


# y_pred_new = y_pred[:,:-1].copy()
# y_pred_new[:,-1] += y_pred[:,-1].copy()


#print(confusion_matrix(np.argmax(labels_new_val,axis=1), np.argmax(y_pred,axis=1)))

#print(precision_recall_fscore_support(np.argmax(labels_new_val,axis=1), np.argmax(y_pred,axis=1), average=None))

a, b, c = roc_curve(labels_new_val[:,1], y_pred[:,1])

# plt.figure()
# plt.plot(a,b,label="17")
# plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
# plt.axis("square")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("One-vs-Rest ROC curves:\nEB vs the rest")
# plt.legend()
#plt.savefig("ROC_fluxbkg_vs_fluxonly.png")


thr = 20


other = y_pred[y_pred[:,1]<c[thr]]
eb = y_pred[y_pred[:,1]>=c[thr]]


correct = 0
cont = 0
ebs = 0
missed = 0 
others = 0
pcs_in_eb = 0

for i in range(len(y_pred)):
  if y_pred[i,1] < c[thr] and labels_new_val[i,1] == 0:
    correct = correct + 1
    others = others + 1
  elif y_pred[i,1] >= c[thr] and labels_new_val[i,1] == 1:
    correct = correct + 1
    ebs = ebs + 1
  elif y_pred[i,1] >= c[thr] and labels_new_val[i,1] == 0:
    cont = cont + 1
    if labels_new_val[i,0] == 1:
        pcs_in_eb = pcs_in_eb + 1
    
  elif y_pred[i,1] < c[thr] and labels_new_val[i,1] == 1:
    missed = missed + 1


accuracy = ebs/len(labels_new_val[labels_new_val[:,1]==1])
contamination = cont/(len(eb))
print(f"percent of ebs found: {accuracy}, contamination: {contamination}, ebs found: {ebs}, ebs missed: {missed}, false positives: {cont}")

print(f"% of FP and PCs removed: {(others)*100/(len(y_true[labels_new_val[:,0]==0]))}")
print(f"num removed: {others}")
print(f"% planets discarded as ebs: {pcs_in_eb/len(y_true[labels_new_val[:,0]==1])}")
print(f"planets discarded as ebs: {pcs_in_eb}")

# my_score = []
# max_score = -1
# for thr in c:
#     #for thr2 in c2:

#     ebs = len(np.where((y_pred[:,1] >= thr)&(labels_new_val[:,1] == 1))[0])
#     missed = len(np.where((y_pred[:,1] < thr)&(labels_new_val[:,1] == 1))[0])
#     pcs_in_eb = len(np.where((y_pred[:,1] >= thr)&(labels_new_val[:,0] == 1))[0])
#     score = ebs/len(np.where(labels_new_val[:,1] == 1)[0])-2.5*(pcs_in_eb/len(np.where(labels_new_val[:,0] == 1)[0]))#-(missed/len(np.where(labels_new_val[:,1] == 1)[0]))
#     my_score.append(score)
#     if score > max_score:
#         max_score = score
#         best_c = thr
#         best_ebs = ebs
#         best_missed = missed
#         best_pcs_in_eb = pcs_in_eb


#     #plt.plot(thr,ebs-pcs_in_eb-missed,".")

# plt.figure()
# plt.plot(my_score)
# print(f"found ebs: {best_ebs}, missed ebs: {best_missed}, pcs removed: {best_pcs_in_eb}")    

# a2, b2, c2 = roc_curve(labels_new_val[:,0], y_pred_new[:,0])

# my_score2 = []
# max_score2 = -100
# best_pcs_in_eb2 = pcs_in_eb
# for thr2 in c2:
#     #for thr2 in c2:
    
#     #ebs2 = len(np.where((y_pred_new[:,1] >= best_c)&(labels_new_val[:,1] == 1)&(y_pred_new[:,0] <= thr2))[0])+len(np.where((y_pred_new[:,1] >= best_c)&(labels_new_val[:,1] == 1)&(y_pred_new[:,0] > thr2))[0])
#     #ebs2 = len(np.where((y_pred_new[:,1] >= best_c)&(labels_new_val[:,1] == 1)&(y_pred_new[:,0]<=thr2))[0]) #+ len(np.where((y_pred_new[:,1] < best_c)&(labels_new_val[:,1] == 1)&(y_pred_new<=thr2))[0])
#     #missed2 = len(np.where((y_pred_new[:,1] < best_c)&(labels_new_val[:,1] == 1)))+len(np.where((y_pred_new[:,1] >= best_c)&(labels_new_val[:,1] == 1)&(y_pred_new[:,0]>thr2))[0])#&(y_pred_new[:,0]>thr2))[0])#+len(np.where((y_pred_new[:,1] < best_c)&(labels_new_val[:,1] == 1)&(y_pred_new[:,0]<=thr2))[0])
#     #pcs_in_eb2 = len(np.where((y_pred_new[:,1] >= best_c)&(labels_new_val[:,0] == 1)&(y_pred_new[:,0]<=thr2))[0])#+len(np.where((y_pred_new[:,1] < best_c)&(labels_new_val[:,0] == 1)&(y_pred_new[:,0]  thr2))[0])
#     pcs_in_eb2 = len(np.where((labels_new_val[:,0]==1)&(y_pred_new[:,1]>=best_c)&(y_pred_new[:,0]<=thr2)))
#     ebs2 = len(np.where((labels_new_val[:,1]==1)&(y_pred_new[:,1]>=best_c)&(y_pred_new[:,0]<=thr2)))
#     score2 = ebs2/len(np.where(labels_new_val[:,1] == 1)[0])-(pcs_in_eb2/len(np.where(labels_new_val[:,0] == 1)[0]))#-(missed2/len(np.where(labels_new_val[:,1] == 1)[0]))
#     my_score2.append(score2)
#     if score2 > max_score2:
#     #if pcs_in_eb2 < best_pcs_in_eb2:
#         max_score2 = score2
#         best_c2 = thr2
#         best_ebs2 = ebs2
#         best_missed2 = missed2
#         best_pcs_in_eb2 = pcs_in_eb2
        
        
# plt.figure()
# plt.plot(my_score2)
# print(f"found ebs: {best_ebs2}, missed ebs: {best_missed2}, pcs removed: {best_pcs_in_eb2}") 

In [ ]:
labels_new_val = y_true4[:,:-1].copy()
labels_new_val[:,-1] += y_true4[:,-1].copy()

# pc_as_eb_index = []
# eb_as_eb_index = []

# best_c2 = 3e-7

# pc_as_pc_index = np.where((y_pred[:,1] < best_c)&(labels_new_val[:,0] == 1)|(y_pred[:,1] >= best_c)&(labels_new_val[:,0] == 1)&(y_pred[:,0] < best_c2))#&(y_pred_new[:,0] > best_c2)) 

# pc_as_eb_index = np.where((y_pred[:,1] >= best_c)&(labels_new_val[:,0] == 1)&(y_pred[:,0] < best_c2)) 

# eb_as_eb_index = np.where((y_pred[:,1] >= best_c)&(labels_new_val[:,1] == 1)&(y_pred[:,0] < best_c2))

# eb_as_pc_index = np.where((y_pred[:,1] < best_c)&(labels_new_val[:,1] == 1)|(y_pred[:,1] >= best_c)&(labels_new_val[:,1] == 1)&(y_pred[:,0] < best_c2))#&(y_pred_new[:,0] > best_c2))

# other = np.where((y_pred_new[:,1] >= best_c)&(y_pred_new[:,0] >= best_c2)) 

#num = 5e-11

pc_as_eb_index = np.where((y_pred4[:,1] >= best_c)&(labels_new_val[:,0]==1))#&(y_pred[:,0]<num))
eb_as_eb_index = np.where((y_pred4[:,1] >= best_c)&(labels_new_val[:,0]==0))#&(y_pred[:,0]<num))

found_pc = np.where((y_pred4[:,1] < best_c)&(labels_new_val[:,0]==1))#&(y_pred[:,0]<num))
eb_as_pc_index = np.where((y_pred4[:,1] < best_c)&(labels_new_val[:,0]==0))#&(y_pred[:,0]<num))
#found_eb = np.where((y_pred4[:,1] >= best_c)&(labels_new_val[:,1]==1))#&(y_pred[:,0]<num))


#fp_as_eb_index = []

# for i in range(len(y_true)):
#     if np.argmax(labels_new_val,axis=1)[i] == 0 and np.argmax(y_pred,axis=1)[i] == 1:
#         pc_as_eb_index.append(i)
#     if np.argmax(labels_new_val,axis=1)[i] == 1 and np.argmax(y_pred,axis=1)[i] == 1:
#         eb_as_eb_index.append(i)
# #     if np.argmax(labels_new_val,axis=1)[i] == 2 and np.argmax(y_pred,axis=1)[i] == 1:
# #         fp_as_eb_index.append(i)
    

pc_as_eb_score = y_pred[pc_as_eb_index]

pc_as_eb_tics = tics[pc_as_eb_index]

#mask = np.where(pc_as_eb_score[:,0]>1e-2)

#pc_as_eb_tics = pc_as_eb_tics[mask]


print("unique pcs as ebs (tics): ", len(np.unique(pc_as_eb_tics)))

eb_as_eb_score = y_pred[eb_as_eb_index]
eb_as_eb_tics = tics[eb_as_eb_index]

#mask = np.where(eb_as_eb_score[:,1]>=best_c)

#eb_as_eb_tics = eb_as_eb_tics[mask]

print("unique ebs as ebs (tics): ", len(np.unique(eb_as_eb_tics)))

# fp_as_eb_score = y_pred[fp_as_eb_index]
# fp_as_eb_tics = tics[fp_as_eb_index]

# mask = np.where((fp_as_eb_score[:,1]>=best_c)&(fp_as_eb_score[:,2]>0))

# fp_as_eb_tics = fp_as_eb_tics[mask]

# print("unique fp as eb (tics): ", len(np.unique(fp_as_eb_tics)))
eb_as_pc_score = y_pred[eb_as_pc_index]
eb_as_pc_tics = tics[eb_as_pc_index]

print("unique ebs as pc (tics): ", len(np.unique(eb_as_pc_tics)))

pc_as_pc_score = y_pred[found_pc]
pc_as_pc_tics = tics[found_pc]

print("unique pc as pc (tics): ", len(np.unique(pc_as_pc_tics)))



np.where((y_pred4[:,1] >= best_c)&(labels_new_val[:,0]==1))

In [ ]:
print(len(np.unique(tics)))
count=0
for tic in np.unique(eb_as_pc_tics):
    if tic in pc_as_pc_tics:
        count+=1


In [ ]:
np.unique(pc_as_eb_tics)

## Make current vs cnn top 500

In [ ]:
#planet_score = y_pred[:,0]

df = pd.DataFrame({"sector":secs[:,0],"tic":tics[:,0],"true":y_true[:,1],"pred":y_pred[:,1]})

top_500_cnn = []
top_500_current = []

files = glob.glob("top_1000_tic_ids/*")

for file in files:
    
    sector = int(file[30:-4])
    
#     for i in pd.read_csv(file)["db_count_weighted"]:
#         current_top_1000_scores.append(np.max(eval(i)))
    
    df_sec = df[df['sector'] == sector]
    
    df_sec_sorted  = df_sec.sort_values('pred', ascending=False)
    
    df_sec_sorted_unique = df_sec_sorted.drop_duplicates(subset='tic', keep="first")
    
    num = len(df_sec_sorted_unique)
    
    num=500
    
    df_sec_sorted_unique_crop = df_sec_sorted_unique[:num]
    
    current_top_1000 = pd.read_csv(file)
    
    current_top_crop = current_top_1000#[:num]
    current_top_crop_in_df = df_sec_sorted_unique[df_sec_sorted_unique["tic"].isin(current_top_crop["TIC_ID"])][:num]
    
    if len(df_sec_sorted_unique) > 0 :
        top_500_cnn.append(df_sec_sorted_unique_crop["true"])
        top_500_current.append(current_top_crop_in_df["true"])
        
        

In [ ]:
# precision, recall metrics for real planets
# recovery rate as a function of rank

from sklearn import metrics

fig , axs = plt.subplots(1,2)

cnn_scores = top_500_cnn[0]
current_scores = top_500_current[0]

ranks = np.arange(1, len(current_scores))
print(len(current_scores))
recs = np.zeros(len(ranks))
precs = np.zeros(len(ranks))
for i, rank in enumerate(ranks):
    prec = metrics.precision_score(current_scores[:rank], np.ones(rank), average="binary")  # taking the top k to be positive predictions
    precs[i] = prec
    rec = np.sum(current_scores[:rank]) / np.sum(current_scores)
    recs[i] = rec

line, = axs[0].plot(ranks, recs, linestyle = 'solid',color="blue",label="current")  #, total={len(cls_df)}")
axs[1].plot(ranks, precs, linestyle = 'solid',color="blue")

ranks = np.arange(1, len(cnn_scores))
print(len(cnn_scores))
recs = np.zeros(len(ranks))
precs = np.zeros(len(ranks))

for i, rank in enumerate(ranks):
    prec = metrics.precision_score(cnn_scores[:rank], np.ones(rank), average="binary")  # taking the top k to be positive predictions
    precs[i] = prec
    rec = np.sum(cnn_scores[:rank]) / np.sum(cnn_scores)
    recs[i] = rec

line, = axs[0].plot(ranks, recs, linestyle = 'solid',color="red",label="CNN")  #, total={len(cls_df)}")
axs[1].plot(ranks, precs, linestyle = 'solid',color="red") 
axs[0].legend()
    


# formatting
axs[0].set_title("Fractional Recovery Rate")
axs[0].set_xlabel("Rank")
axs[0].set_ylabel("Recall")
# axs[0].set_xlim(-0.05,1.05)
#axs[0].set_ylim(-0.05, 0.7)
#axs[0].legend(title="Candidate")
axs[0].set_aspect(1./axs[0].get_data_ratio())

axs[1].set_title("Precision")
axs[1].set_xlabel("Rank")
axs[1].set_ylabel("Precision")
# axs[1].set_xlim(-0.05,1.05)
#axs[1].set_ylim(-0.05, 1)
#axs[1].legend(title="Candidate")
axs[1].set_aspect(1./axs[1].get_data_ratio())


plt.tight_layout()
plt.show()

### top_500_cnn